### TF test

In [ ]:
#coding=utf-8
__author__ = 'Marshall'
import tensorflow as tf
import numpy as np  #科学计算
# 神经网络学习一元一次函数的例子
# create data
x_data = np.random.rand(101).astype(np.float32) #生成100随机数列，取随机数值并转换为float32格式
y_data = x_data*0.1 + 0.3 #真实的答案W=0.1，b=0.3  y=Weights * x + biases

In [ ]:
import tensorflow as tf

In [ ]:
#创建模型建构
Weights = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) #大写因为可能是矩阵，生成-1到1的随机数，但W目前是1维
biases = tf.Variable(tf.zeros([1])) #定义初始值为0的1维标量

y = Weights*x_data + biases

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
#计算误差loss
loss = tf.reduce_mean(tf.square(y-y_data)) #计算 y 和 实际额y_data 的误差 通过平均方差计算


In [ ]:
#传播误差
optimizer = tf.train.GradientDescentOptimizer(0.5) #优化器反向传递误差，使用梯度下降法，学习效率0.5，[0,1]
train = optimizer.minimize(loss)#然后用optimizer更新参数

In [ ]:
#开始训练
# init = tf.initialize_all_variables() # tf 马上就要废弃这种写法
init = tf.global_variables_initializer()  # 替换成这样就好，初始化之前定义的变量

# 创建会话
sess = tf.Session()
sess.run(init)       #类似于指针，激活

for step in range(201): #训练201步骤
    sess.run(train) #真正开始训练
    if step % 20 == 0: #每隔上20步打印一下
        print(step, sess.run(Weights), sess.run(biases)) #打印也要用sess.run()
writer=tf.summary.FileWriter('./log/ex2',sess.graph)
writer.flush()
writer.close()

0 [0.60092723] [0.02559315]
20 [0.25188294] [0.21603468]
40 [0.1461498] [0.27448705]
60 [0.11402268] [0.29224786]
80 [0.1042608] [0.29764453]
100 [0.10129465] [0.29928428]
120 [0.10039337] [0.29978254]
140 [0.10011952] [0.29993394]
160 [0.10003632] [0.29997993]
180 [0.10001101] [0.29999393]
200 [0.10000335] [0.29999816]


### GPU / CPU TEST

In [ ]:
from keras import backend as K
K.tensorflow_backend.get_available_gpus()

Using TensorFlow backend.


AttributeError: ignored

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 109630848992584417
]


In [ ]:
#UNI CPU
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8224751922808633610
]


In [ ]:
#UNI GPU k200
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16076076796275673
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6786838037
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5578966931307150928
physical_device_desc: "device: 0, name: Quadro K5200, pci bus id: 0000:02:00.0, compute capability: 3.5"
]


In [ ]:
#UNI GPU p4000
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12136897180997295179
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6697371566
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11667496954427207623
physical_device_desc: "device: 0, name: Quadro P4000, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


In [ ]:
#Colab GPU Tesla T4
from tensorflow.python.client import device_lib
if __name__ == "__main__":
    print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2219057776319970633
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8985708448985471061
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5216652745800661483
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14648777152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5226210414079972320
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


### MultiProcessing

In [ ]:
import timeit
timeit.timeit(lambda:"-".join(map(str,[1,2,3])), number=10)

1.2512000466813333e-05

In [ ]:
import multiprocessing as mp
print('CPU numbers:',mp.cpu_count())
# uni 12

CPU numbers: 12


In [ ]:
#colab 2
import multiprocessing as mp
print('CPU numbers:',mp.cpu_count())

CPU numbers: 2


In [ ]:
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = mp.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
#apply_by_multiprocessing(fullset['Text'], process_text, workers=cores)

In [ ]:
def worker():
    """worker function"""
    print('Worker')
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = mp.Process(target=worker)
        jobs.append(p)
        p.start()

'__main__'

### Disk storage

In [ ]:
import ctypes
import os
import platform
import sys

In [ ]:
platform.system()

'Windows'

In [ ]:
def get_free_space(folder):
    free_bytes = ctypes.c_ulonglong(0)
    ctypes.windll.kernel32.GetDiskFreeSpaceExW(ctypes.c_wchar_p(folder)), None, None, ctypes.pointer
    return free_bytes.values/1024/1024/1024

In [ ]:
["C:Users/lip17add/{0}.csv","X:/Creative_Media/Data/2018_19/LIP17ADD/X_Download/']

In [ ]:
import sys
sys.path

In [ ]:
import tensorflow as tf
tf.__file__